In [1]:
import string
import fasttext
import math
import copy
import re
import numpy as np
import pandas as pd
import os
from nltk import TweetTokenizer
from nltk.tokenize.stanford import StanfordTokenizer
from scipy import spatial
from scipy.spatial import distance
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
cachedStopWords = stopwords.words("english")
CLEANR = re.compile('<.*?>')


[nltk_data] Downloading package stopwords to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tokenization

In [2]:
def remove_punctuation(text, punct_list):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()


def remove_html_tags(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

def tokenize(sentence, to_lower=True, tknzr=TweetTokenizer()):
    """Arguments:
        - tknzr: a tokenizer implementing the NLTK tokenizer interface
        - sentence: a string to be tokenized
        - to_lower: lowercasing or not
    """
    sentence = sentence.strip()
    sentence = ' '.join([format_token(x) for x in tknzr.tokenize(sentence)])
    if to_lower:
        sentence = sentence.lower()
    # replace urls by <url>
    sentence = re.sub(
        '((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', sentence)
    # replace @user268 by <user>
    sentence = re.sub('(\@[^\s]+)', '', sentence)

    filter(lambda word: ' ' not in word, sentence)

    #remove single letter words
    sentence = ' '.join([w for w in sentence.split() if len(w) > 1])

    sentence = remove_html_tags(sentence)
    sentence = remove_punctuation(sentence, list(string.punctuation))
    sentence = ' '.join([word for word in sentence.split()
                        if word not in cachedStopWords])
    return sentence

def format_token(token):
    """"""
    if token == '-LRB-':
        token = '('
    elif token == '-RRB-':
        token = ')'
    elif token == '-RSB-':
        token = ']'
    elif token == '-LSB-':
        token = '['
    elif token == '-LCB-':
        token = '{'
    elif token == '-RCB-':
        token = '}'
    return token

### Nearest neighbor

In [3]:
def NN(model,line,K):
    return model.get_nearest_neighbors(line, k=K)


### Cosine Similarity

In [4]:
def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	
	return dot_product / (norm_a * norm_b)

## Main

### Load data

In [5]:
# load model
model = fasttext.load_model('fp_bigrams_unsupervised.bin')

In [6]:
# Load seed set
text_file = open("seed_set.txt", "r")
no_str = text_file.read()
text_file.close()
# make a list
seed_set = no_str.split("\n")

# Load Labels
df_seed = pd.read_csv("seed_set.csv")
Y = df_seed["label"].tolist()
print(Y)

# keep a copy
seed_set_copy = copy.deepcopy(seed_set)
#cleanup
temp = []
for comment in seed_set:
    tok_comment = tokenize(comment)
    temp.append(tok_comment)
seed_set = temp

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [7]:
# Load corpus
text_file2 = open("corpus.txt", "r")
no_str2 = text_file2.read()
text_file2.close()
# make a list
corpus = no_str2.split("\n")

# keep a copy
corpus_copy = copy.deepcopy(corpus)
# cleanup
temp = []
for comment in corpus:
    tok_comment = tokenize(comment)
    temp.append(tok_comment)
corpus = temp


### Nearest Neighbor sets

In [8]:
# nearest neigbors for seed set, setting k = 20
NN_seed_set = []
for comment in seed_set:
    NN_seed_set.append(NN(model,comment,20))

In [9]:
# nearest neigbors for unlabeled corpus, setting k = 20
NN_corpus = []
for comment in corpus:
    NN_corpus.append(NN(model,comment,20))

### NN Sampling by finding distance

In [10]:
def intersection_score(words, lst1, lst2):

    # words contains all the words in the corpus
    v1 = []
    v2 = []

    l1 = [x[1] for x in lst1]
    l2 = [x[1] for x in lst2]

    l10 = [x[0] for x in lst1]
    l20 = [x[0] for x in lst2]

    for i in range(len(words)):
        try:
            v1.append(l10[l1.index(words[i])])
        except:
            v1.append(0)

    for i in range(len(words)):
        try:
            v2.append(l20[l2.index(words[i])])
        except:
            v2.append(0)
    
    return cos_sim(np.array(v1), np.array(v2))


In [11]:
print(seed_set[0])
print(corpus[0])
print(intersection_score(model.words, NN_seed_set[0],NN_corpus[0]))

kisan andolan propagation necessary take matter poor people suffering kisan andolan lost livelihood
choukidar chor jasoos 2024 choukidar chor gol
0.04291699652667399


In [12]:
# words: model.words, d1: NN_seed_set, l2: NN_corpus_line, g_seed_set: seed_set, g_corpus_comment: corpus_line, Y:labels
def expand_label(words,d1, l2, g_seed_set, g_corpus_comment, Y):

    scores = [intersection_score(words,i, l2) for i in d1]
    maxpos = scores.index(max(scores))

    try:
        if(Y[maxpos] == 0):
            Y.insert(0,0)
            d1.insert(0,l2)
            g_seed_set.insert(0,g_corpus_comment)
        elif(Y[maxpos] == 2):
            Y.append(2)
            d1.append(l2)
            g_seed_set.append(g_corpus_comment)
        elif(Y[maxpos] == 1):
            d1.insert(Y.index(1),l2)
            g_seed_set.insert(Y.index(1),g_corpus_comment)
            Y.insert(Y.index(1),1)
    except:
        pass


In [13]:
# expand seed set along with labels
Y_expanded = Y
NN_seed_set_expanded = NN_seed_set

# expand by 20 comments
for i in range(20): 
    expand_label(model.words,NN_seed_set_expanded,NN_corpus[i+1000],seed_set_copy, corpus_copy[i+1000], Y_expanded)

In [14]:
# save as txt
with open('seed_set_expanded.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(seed_set_copy))

Y_expanded_string = map(str, Y_expanded)
with open('seed_set_expanded_labels.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(Y_expanded_string))